In [1]:
import numpy as np
import random

# %pip install tensorflow-gpu

# %pip install cuda

# %pip install cudnn


In [7]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/CPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Run on the GPU
c = tf.matmul(a, b)
print(c)



Num GPUs Available:  0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.config.list_physical_devices('GPU')

from tensorflow.python.client import device_lib

device_lib.list_local_devices()

tf.test.is_built_with_cuda()

tf.debugging.set_log_device_placement(True)

Num GPUs Available:  0


2022-11-13 17:55:19.018477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 17:55:19.026377: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-13 17:55:19.028036: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-11-13 17:55:19.028069: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required

In [ ]:
import os
import sys
my_lib_path = os.path.abspath(
    '/home/mai.kassem/Documents/ML701-AilmentClassification/')
sys.path.append(my_lib_path)
import lstm_builder

In [ ]:
n = 10  # population size
m = 7  # good bees
e = 3  # the elite bees
num = 2  # numbers of the LSTM parameters - number of epochs and number of units
nep = 4
nsp = 2
ngh = 1  # neighborhood size
opt = np.zeros([1, num+1])  # the result (i.e., the optimal bee)
# second parameter is the LSTM parameters + the accuracy
population = np.zeros([n, num+1])
good = np.zeros([m-e, num+1])
elite = np.zeros([e, num+1])

epochs_start = 10
epochs_end = 50
units_start = 8
units_end = 128


In [ ]:
def generate_bee(index):
    # generate the parameters using uniform distribution function
    # first parameter
    population[index][0] = int(random.uniform(epochs_start, epochs_end))
    # second parameter
    population[index][1] = int(random.uniform(units_start, units_end))


In [ ]:
def evaluate(bee):
    # train NN and calculate the accuracy
    # accuracy = random.randint(90, 99)
    model = lstm_builder.LSTM(num_epochs=int(bee[0]), num_units=int(bee[1]))
    predicted_target, actual_target, average_accuracy = lstm_builder.evaluate_model(
        model)
    bee[num] = average_accuracy
    return bee


In [ ]:
def sort_bees(unsorted):
    return np.array(sorted(unsorted, key=lambda x: x[num], reverse=True))


In [ ]:
def local_search(bee, st, amount):
    new_bees = np.zeros([amount, num+1])
    for i in range(amount):
        # generate new bee in neighborhood of current bee
        nbee = [0, 0, 0]
        # first parameter
        nbee[0] = int(random.uniform(bee[0]-ngh, bee[0]+ngh))
        # second parameter
        nbee[1] = int(random.uniform(bee[1]-ngh, bee[1]-ngh))
        # evaluate new bee
        nbee = evaluate(nbee)
        # if the evaluation of new bee is better than original, keep it else save the original
        if nbee[num] > bee[num]:
            new_bees[i] = nbee
        else:
            new_bees[i] = bee

    new_bees = sort_bees(new_bees)
    return new_bees[0]


In [ ]:
for i in range(n):
    generate_bee(i)
    # evaluate the population
    population[i] = evaluate(population[i])

# sort the population decesntly based on the evaluation value (i.e., accuracy)
population = sort_bees(population)

# opt=first bee in the population
opt = population[0]


In [ ]:
print(population)
print()
print(opt)


In [ ]:
maxIter = 10
for iter in range(maxIter):
    print("Iteration==========================", iter)
    print(population)

    for i in range(e):
        # call local search for elite and update the population
        population[i] = local_search(population[i], 'elite', nep)
    for j in range(i, m):
        # call local search for the remaining good bees and update the population
        # i = i+1
        population[j] = local_search(population[j], 'good', nsp)
        # generate the remaining of the population randomly and evaluate
        # i = i+1
        # while (i <= n):
        #     generate_bee(i)
        #     # evaluate the population
        #     population[i] = evaluate(population[i])

    for k in range(j, n):
        # generate the remaining of the population randomly and evaluate
        generate_bee(k)
        # evaluate the population
        population[k] = evaluate(population[k])

    # sort the population
    population = sort_bees(population)
    # opt= the first bee in the population
    opt = population[0]


In [ ]:
print(population)
print()
print(opt)
